In [1]:
import pandas as pd, numpy as np
import sklearn
import scipy.sparse
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from bs4 import BeautifulSoup
import re
from sklearn.metrics import f1_score

In [ ]:
data = pd.read_csv('../data/raw_data/kaggle/train.csv')
labels=(data["toxic"] | data["severe_toxic"] | data["obscene"] | data["threat"] | data["insult"] | data["identity_hate"])
# labels=data["Labels"]
train_data = data[0:100000]
test_data = data[100001:]
train_labels = labels[0:100000]
test_labels = labels[100001:]

corpus = data["comment_text"]

In [ ]:
vectorizer = CountVectorizer(strip_accents='unicode',lowercase=True,stop_words='english', analyzer="word",token_pattern=r'\w{1,}')
X = vectorizer.fit_transform(corpus)
train_feature_vectors_sparse = X[0:100000]
test_feature_vectors_sparse = X[100001:]

In [ ]:
def cross_val_tf_idf(tfidf_transformer):
    X_train_tfidf = tfidf_transformer.fit_transform(train_feature_vectors_sparse)
    X_test_tfidf = tfidf_transformer.transform(test_feature_vectors_sparse)

    # TRAINING

    clf = MultinomialNB()
    clf.fit(X_train_tfidf, train_labels)
    
    # PREDICTION
    predictions = clf.predict(X_test_tfidf)


    # PERFORMANCE METRICS

    confmat = confusion_matrix(test_labels, predictions)

    precision = confmat[1,1]/(confmat[1,1] + confmat[1,0])
    recall = confmat[1,1]/(confmat[1,1] + confmat[0,1])
    fScore = f1_score(test_labels, predictions, average='weighted')
    ccr = (confmat[0,0] + confmat[1,1])/(sum(sum(confmat)))

    print("Precision = ",precision)
    print("Recall = ",recall)
    print("F_score = ",fScore)
    print("CCR = ",ccr)
    print()
    print("Confusion Matrix: \n", confmat)
    
    return [precision,recall, fScore, ccr]

In [ ]:
fsc = []
prec = []
recall = []
ccr = []

tfidf_transformer = TfidfTransformer(norm=None, use_idf=False, smooth_idf=True, sublinear_tf=False)
res = cross_val_tf_idf(tfidf_transformer)

fsc.append(res[2])
prec.append(res[0])
recall.append(res[1])
ccr.append(res[3])

tfidf_transformer = TfidfTransformer(norm=None, use_idf=True, smooth_idf=True, sublinear_tf=False)
res = cross_val_tf_idf(tfidf_transformer)

fsc.append(res[2])
prec.append(res[0])
recall.append(res[1])
ccr.append(res[3])
